In [1]:
import tweepy
import numpy as np
import pandas as pd
import datetime
import pytz

In [2]:
# consumer_key = "0EWVmQYSBV9swiZxUKSXyXmel" #Your API/Consumer key 
# consumer_secret = "vHqqKvLoSSwcPmF1ryzMxbhYO0GqGqe6BTPfa1Q3K0qwvAVoUw" #Your API/Consumer Secret Key
# access_token = "978638940-on0RJCmXAEUCiUw5uioUGKaGKSm4wpMN83W01Sbp"    #Your Access token key
# access_token_secret = "wT7wAiMd5TBer1NIE5KSGHPANWjMYxWkBitlGQJsX5FdP" #Your Access token Secret key
# bearer_token = "AAAAAAAAAAAAAAAAAAAAAB8omgEAAAAA2mVsfTgEY2d3NdHZeMNEHnx%2BHJ0%3DrpqCTFfg5mfIIP8VSIhFN2Cn4G70giXXM9nQU1pMXXSTZEgEfN"

# # # Authenticate with Twitter API
# # auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# # auth.set_access_token(access_token, access_token_secret)

# # client = tweepy.Client(bearer_token=bearer_token,
# #                        consumer_key=consumer_key,
# #                        consumer_secret=consumer_secret,
# #                        access_token=access_token,
# #                        access_token_secret=access_token_secret,
# #                        wait_on_rate_limit=True)

# # user = client.get_user(username="farrelium")


# # # Create API object
# # api = tweepy.API(auth)

# # screen_name = 'farrelium'  # Replace with the username of the user whose tweets you want to retrieve
# # tweet_count = 10  # Number of tweets to retrieve

# # # Get user tweets
# # tweets = api.user_timeline(screen_name=screen_name, count=tweet_count)

# # # Extract tweet text
# # for tweet in tweets:
# #     print(tweet.text)

In [4]:
consumer_key = "0EWVmQYSBV9swiZxUKSXyXmel" #Your API/Consumer key 
consumer_secret = "vHqqKvLoSSwcPmF1ryzMxbhYO0GqGqe6BTPfa1Q3K0qwvAVoUw" #Your API/Consumer Secret Key
access_token = "978638940-on0RJCmXAEUCiUw5uioUGKaGKSm4wpMN83W01Sbp"    #Your Access token key
access_token_secret = "wT7wAiMd5TBer1NIE5KSGHPANWjMYxWkBitlGQJsX5FdP" #Your Access token Secret key
bearer_token = "AAAAAAAAAAAAAAAAAAAAAB8omgEAAAAA2mVsfTgEY2d3NdHZeMNEHnx%2BHJ0%3DrpqCTFfg5mfIIP8VSIhFN2Cn4G70giXXM9nQU1pMXXSTZEgEfN"

# Pass in our twitter API authentication key
auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret,
    access_token, access_token_secret
)

#Instantiate the tweepy API
api = tweepy.API(auth, wait_on_rate_limit=True)

In [5]:
# Error handling
# Expired token -> 401 Unauthorized
# Wrong username -> 404 Not Found
# Private account -> 401 Unauthorized
# Have no tweets -> 666 No Tweets
# Tweets < 20

username = "farrelium"

# Define tweets will be taken from 2 years from now
utc = pytz.UTC
endDate = datetime.datetime.now()
startDate = endDate - datetime.timedelta(days=2*365)
endDate = utc.localize(endDate) 
startDate = utc.localize(startDate)

tweets = []

try:
    # The number of tweets we want to retrieved from the user in a specific period of time
    temp_tweets = api.user_timeline(screen_name=username)
    for tweet in temp_tweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)
    
    if len(tweets) > 0:
        while (temp_tweets[-1].created_at > startDate):
            temp_tweets = api.user_timeline(screen_name=username, max_id = temp_tweets[-1].id)
            for tweet in temp_tweets:
                if tweet.created_at < endDate and tweet.created_at > startDate and len(tweets) < 40: # Max 200 tweets
                    tweets.append(tweet)

        #Pulling Some attributes from the tweet
        #attributes_container = [[tweet.created_at,tweet.favorite_count,tweet.source,tweet.text] for tweet in tweets]
        attributes_container = [[tweet.created_at,tweet.text] for tweet in tweets]

        #Creation of column list to rename the columns in the dataframe
        #columns = ["Date Created", "Number of Likes", "Source of Tweet", "Tweet"]
        columns = ["date_created", "tweet"]

        #Creation of Dataframe
        crawl_df = pd.DataFrame(attributes_container, columns=columns)
    else:
        raise Exception("666 No Tweets")
except BaseException as e:
    # Kalau error, return e (di flask)
    print(str(e))

In [6]:
tweets[0].text

'Sometimes shit happens.'

In [7]:
attributes_container

[[datetime.datetime(2023, 4, 4, 13, 29, 44, tzinfo=datetime.timezone.utc),
  'Sometimes shit happens.'],
 [datetime.datetime(2023, 4, 3, 14, 48, 16, tzinfo=datetime.timezone.utc),
  'Coca cola is tasty 😂'],
 [datetime.datetime(2023, 4, 2, 20, 5, 33, tzinfo=datetime.timezone.utc),
  '@ChelseaFC Leicester sama Tottenham lagi open slot tuh om Potter'],
 [datetime.datetime(2023, 4, 1, 17, 50, 1, tzinfo=datetime.timezone.utc),
  'Potter ni apa ya'],
 [datetime.datetime(2023, 3, 30, 12, 34, 23, tzinfo=datetime.timezone.utc),
  'Character developmentnya Canute sama Thorfinn nih kaya kebalikan ya, ga sabar liat mereka ketemu next ep 👀'],
 [datetime.datetime(2023, 3, 22, 16, 24, 18, tzinfo=datetime.timezone.utc),
  'Ga expect ternyata Summer Strike sebagus inii'],
 [datetime.datetime(2023, 3, 18, 19, 51, 8, tzinfo=datetime.timezone.utc),
  "@DhruvrCFC @ConnCFC He can't even park a bus. The subs go the field with no clue"],
 [datetime.datetime(2023, 3, 18, 19, 42, 32, tzinfo=datetime.timezone.ut

In [8]:
crawl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   date_created  40 non-null     datetime64[ns, UTC]
 1   tweet         40 non-null     object             
dtypes: datetime64[ns, UTC](1), object(1)
memory usage: 768.0+ bytes


In [9]:
crawl_df.tail()

,date_created,tweet
35,2022-12-31 16:53:40+00:00,@ignatiajssica Yang dipilih pas poseku manyun ...
36,2022-12-20 15:41:15+00:00,"@anyafojawaku Waalaikumsalam, sukses terus kel..."
37,2022-12-20 07:39:38+00:00,"@anyafojawaku Sungkem duluu ada master wkwk, i..."
38,2022-12-19 17:02:01+00:00,Suara hyein ternyata ada merdu-merdunyaa
39,2022-12-19 16:37:12+00:00,"This ""johahandago"" part is so addictivee"


In [10]:
len(crawl_df)

40

In [11]:
# Check if there's min 20 tweets
if len(crawl_df) > 20:
    sample_df = crawl_df.sample(n=20)
else:
    raise Exception("667 Tweets less than 20")
    sample_df = crawl_df.copy()
    
sample_df

,date_created,tweet
31,2023-01-03 09:20:07+00:00,"@falqiyusuf ashiapp, resolusi 2023 ya bang?"
21,2023-03-05 10:57:53+00:00,RT @denicmarko: Me and my Udemy courses: https...
0,2023-04-04 13:29:44+00:00,Sometimes shit happens.
33,2023-01-02 14:55:02+00:00,Keren sih haerin nyanyi bisa kaya centil centi...
14,2023-03-10 13:04:19+00:00,"Based on plot, kimi no na wa masih terlalu sul..."
38,2022-12-19 17:02:01+00:00,Suara hyein ternyata ada merdu-merdunyaa
35,2022-12-31 16:53:40+00:00,@ignatiajssica Yang dipilih pas poseku manyun ...
20,2023-03-07 23:11:09+00:00,😌 https://t.co/0zrWEFKzGc
26,2023-02-22 15:27:29+00:00,Seketika sadar diri setelah tahu umur chiquita
37,2022-12-20 07:39:38+00:00,"@anyafojawaku Sungkem duluu ada master wkwk, i..."


In [38]:
type(sample_df) == pd.DataFrame

True

## Preprocessing

In [12]:
import re
import preprocessor
import nltk
import goslate
import enchant
from googletrans import Translator
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

en = enchant.Dict("en_US")
idn = []

In [13]:
def casefolding(review):
    review = review.lower()
    return review

def filtering(review):
    review = re.sub(r'@[^\s]+', '', review)  # @username
    review = re.sub(r'#([^\s]+)', '', review)  # hashtag
    review = re.sub(r'https:[^\s]+', '', review)  # URL links
    review = re.sub(r"[.,:;+!\-_<^/=?\"'\(\)\d\*]", " ", review)  # symbol, char
    review = re.sub(r'[^\x00-\x7f]+', '', review)  # non ASCII chars
    review = re.sub(r'\s+', ' ', review)  # duplicate whitespace
    return preprocessor.clean(review)

def tokenizing(review):
    token = nltk.word_tokenize(review)
    return token

def stemming(review):
    factorySt = StemmerFactory()
    stemmer = factorySt.create_stemmer()
    review = stemmer.stem(review)
    return review

def stopWordRemoving(review):
    factorySw = StopWordRemoverFactory()
    stopword = factorySw.create_stop_word_remover()
    review = stopword.remove(review)
    return review

def slangWordConverting(review):
    slangwords = dict()
    with open('slangword-id.txt') as wordfile:
        for word in wordfile:
            word = word.split('=')
            slangwords[word[0]] = word[1].replace('\n', '')

    wordsArray, fixed = review.split(' '), []
    for word in wordsArray:
        if word in slangwords:
            word = slangwords[word]
        fixed.append(word)
        review = ' '.join(fixed)
    return review

def characterRepeating(review):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    tempWord = ''
    for word in review.split(' '):
        if word != '':
            if en.check(word):
                tempWord += word+' '
            elif word in idn:
                tempWord += word+' '
            else:
                tempWord += pattern.sub(r"\1", word) + ' '
    return tempWord


def translating(review):
    gs = goslate.Goslate()
    translatedText = gs.translate(review, 'id')
    return translatedText

def translating2(review):
    translator = Translator()
    translations = translator.translate(review, dest='id')
    return translations.text

def preprocess_tweets(tweet):
    tweet = filtering(str(tweet))
    tweet = casefolding(str(tweet))
    tweet = characterRepeating(str(tweet))
    # tweet = translating(str(tweet))
    tweet = stemming(str(tweet))
    tweet = stopWordRemoving(str(tweet))
    tweet = slangWordConverting(str(tweet))
    # print('Processed: '+str(tweet) + '\n')
    return tweet

In [14]:
sample_df['preprocessed'] = sample_df['tweet'].apply(preprocess_tweets)

In [15]:
sample_df['preprocessed']

31                                 ashiap resolusi bang
21                           saya dan saya udemy kursus
0                                  kadang kotor happens
33    keren sih haerin nyanyi seperti centil centil ...
14    based nyala rencana kim tidak na whatsapp terl...
38                        suara hyein nyata merdu merdu
35                    pilih tidak pose manyun wak je je
20                                                     
26                        sadar diri tahu umur chiquita
37    sungkem dulu kuasa tawa lagu baru newjeans set...
16                                     suzume soal baik
17    kira ini adalah sebuah wakeup panggil untuk ja...
30        jadi eun topped tunggal neraka s in saya buku
1                                coca cola adalah lezat
29                            nadine deh jadi e humoris
24                                                     
5            tidak harap nyata musim panas serang bagus
28                                            bi

In [16]:
result_tweet = " ".join(sample_df['preprocessed'])

In [17]:
result_df = pd.DataFrame({
    'username': [username],
    'tweet': [result_tweet]
})

result_df

,username,tweet
0,farrelium,ashiap resolusi bang saya dan saya udemy kursu...


## Inference

In [20]:
import joblib
import pickle

# Load tfidf
# with open("../personality-evaluator/PPT-09/models/tfidf_2.pickle", "rb") as file:
#     tfidf = pickle.load(file)
    
# Load best model
model = joblib.load("../../PPT-09/models/pipeline.joblib")

In [21]:
X = result_df['tweet']

In [22]:
X

0    ashiap resolusi bang saya dan saya udemy kursu...
Name: tweet, dtype: object

In [23]:
y_pred_proba = model.predict_proba(X)
y_pred_proba

array([[0.112, 0.088, 0.384, 0.344, 0.072]])

In [24]:
list_proba = y_pred_proba[0]
list_class = [0, 1, 2, 3, 4]
list_username = [username] * 5

In [25]:
reshape_df = pd.DataFrame(list(zip(list_username, list_class, list_proba)),
               columns =['user', 'class', 'proba_tweet'])
reshape_df

,user,class,proba_tweet
0,farrelium,0,0.112
1,farrelium,1,0.088
2,farrelium,2,0.384
3,farrelium,3,0.344
4,farrelium,4,0.072


In [26]:
reshape_df.iloc[0]['proba_tweet']

0.112

In [27]:
# Ecek-eceknya dapet data dari frontend berupa json
request = {
    'test_result': [3.75, 2.22, 3.56, 5.00, 6.0]
}

reshape_df['test_result'] = request['test_result']
reshape_df['proba_tweet'] = reshape_df['proba_tweet'] * 10 # Normalisasi
reshape_df

,user,class,proba_tweet,test_result
0,farrelium,0,1.12,3.75
1,farrelium,1,0.88,2.22
2,farrelium,2,3.84,3.56
3,farrelium,3,3.44,5.00
4,farrelium,4,0.72,6.00


In [28]:
reshape_df['inference'] = reshape_df['proba_tweet']*0.6 + reshape_df['test_result']*0.4

In [29]:
reshape_df.head()

,user,class,proba_tweet,test_result,inference
0,farrelium,0,1.12,3.75,2.172
1,farrelium,1,0.88,2.22,1.416
2,farrelium,2,3.84,3.56,3.728
3,farrelium,3,3.44,5.00,4.064
4,farrelium,4,0.72,6.00,2.832


In [30]:
y = reshape_df['inference'].argmax()

In [31]:
result = reshape_df.iloc[y]
result

user           farrelium
class                  3
proba_tweet         3.44
test_result          5.0
inference          4.064
Name: 3, dtype: object

In [32]:
d = {
     0:'Neuroticism',
     1:'Agreeableness',
     2:'Extraversion',
     3:'Openness',
     4:'Conscientiousness'
}

# Return class
d[result['class']]

'Openness'

## Labeling BFI

In [ ]:
# OCEANnumber = {
#     'Extraversion': [1, 6, 11, 16, 21, 26, 31, 36],
#     'Agreeableness': [2, 7, 12, 17, 22, 27, 32, 37, 42],
#     'Conscientiousness': [3, 8, 13, 18, 23, 28, 33, 38, 43],
#     'Neuroticism': [4, 9, 14, 19, 24, 29, 34, 39],
#     'Openness': [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]
# }

# R = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]

In [ ]:
# OCEAN = {
#         'Extraversion': 0,
#         'Agreeableness': 0,
#         'Conscientiousness': 0,
#         'Neuroticism': 0,
#         'Openness': 0
#     }

# bfi_form =  ["h3llatrash", 2, 4, 5, 5, 4, 4, 3, 2, 2, 4, 3, 3, 4, 4, 5, 3, 3, 4, 5, 5, 4, 2, 2, 3, 4, 4, 5, 4, 4, 5, 5, 1, 2, 3, 4, 2, 4, 4, 5, 5, 1, 3, 4, 4]

In [ ]:
# for nomer_soal in range(1, 45):
#     if nomer_soal in OCEANnumber['Extraversion']:
#             if int(bfi_form[nomer_soal]) > 3:
#                 if nomer_soal not in R:
#                     OCEAN['Extraversion'] += 1
                    
# for nomer_soal in range(1, 45):
#     if nomer_soal in OCEANnumber['Agreeableness']:
#             if int(bfi_form[nomer_soal]) > 3:
#                 if nomer_soal not in R:
#                     OCEAN['Agreeableness'] += 1
                    
# for nomer_soal in range(1, 45):
#     if nomer_soal in OCEANnumber['Conscientiousness']:
#             if int(bfi_form[nomer_soal]) > 3:
#                 if nomer_soal not in R:
#                     OCEAN['Conscientiousness'] += 1
                    
# for nomer_soal in range(1, 45):
#     if nomer_soal in OCEANnumber['Neuroticism']:
#             if int(bfi_form[nomer_soal]) > 3:
#                 if nomer_soal not in R:
#                     OCEAN['Neuroticism'] += 1
                    
# for nomer_soal in range(1, 45):
#     if nomer_soal in OCEANnumber['Openness']:
#             if int(bfi_form[nomer_soal]) > 3:
#                 if nomer_soal not in R:
#                     OCEAN['Openness'] += 1

In [ ]:
# OCEAN

In [ ]:
# print(len(OCEANnumber['Extraversion']))
# print(len(OCEANnumber['Agreeableness']))
# print(len(OCEANnumber['Conscientiousness']))
# print(len(OCEANnumber['Neuroticism']))
# print(len(OCEANnumber['Openness']))

In [ ]:
# OCEAN['Extraversion'] /= len(OCEANnumber['Extraversion']) / 10
# OCEAN['Agreeableness'] /= len(OCEANnumber['Agreeableness']) / 10
# OCEAN['Conscientiousness'] /= len(OCEANnumber['Conscientiousness']) / 10
# OCEAN['Neuroticism'] /= len(OCEANnumber['Neuroticism']) / 10
# OCEAN['Openness'] /= len(OCEANnumber['Openness']) / 10

# OCEAN['Extraversion'] = round(OCEAN['Extraversion'], 2)
# OCEAN['Agreeableness'] = round(OCEAN['Agreeableness'], 2)
# OCEAN['Conscientiousness'] = round(OCEAN['Conscientiousness'], 2)
# OCEAN['Neuroticism'] = round(OCEAN['Neuroticism'], 2)
# OCEAN['Openness'] = round(OCEAN['Openness'], 2)

# OCEAN['Extraversion'] = str(OCEAN['Extraversion'])
# OCEAN['Agreeableness'] = str(OCEAN['Agreeableness'])
# OCEAN['Conscientiousness'] = str(OCEAN['Conscientiousness'])
# OCEAN['Neuroticism'] = str(OCEAN['Neuroticism'])
# OCEAN['Openness'] = str(OCEAN['Openness'])

In [ ]:
# OCEAN